In [1]:
import numpy as np
import pandas as pd
import pymc3 as pm
from os.path import join, isfile
from os import listdir
from functools import partial

import glam
import parameter_recovery as pr

%load_ext autoreload
%autoreload 2

/Users/felixmolter/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# Systematic parameter recovery – Hierarchical model, with Metropolis

In [2]:
output_folder = join('results', 'parameter-recovery_hierarchical', 'metropolis')
label = 'hierarchical_metropolis'

## Define parameter ranges from 2011 estimates

In [3]:
bounds = dict(v=[0.00002, 0.00015],
              gamma=[-1, 1],
              s=[0.006, 0.013],
              tau=[0.1, 1.25])

In [4]:
# Determine low, medium, high mean values as 10th, 50th and 90th percentile of these ranges
means = {parameter: np.percentile(np.linspace(*bounds[parameter], 100), [10, 50, 90])
          for parameter in ['v', 'gamma', 's', 'tau']}
means

{'v': array([3.30e-05, 8.50e-05, 1.37e-04]),
 'gamma': array([-8.00000000e-01,  5.55111512e-17,  8.00000000e-01]),
 's': array([0.0067, 0.0095, 0.0123]),
 'tau': array([0.215, 0.675, 1.135])}

In [5]:
sds = dict(v=0.00005,
           s=0.001,
           gamma=0.3,
           tau=0.2)

In [6]:
parameter_info = {parameter: dict(mu=dict(low=means[parameter][0],
                                          medium=means[parameter][1],
                                          high=means[parameter][2]),
                                  sd=sds[parameter],
                                  bounds=tuple(bounds[parameter]))
                  for parameter in ['v', 'gamma', 's', 'tau']}

## Run the recovery in parallel

In [7]:
partial_output_folder = join(output_folder, 'partial')

simulate_group_args = dict(kind='hierarchical',
                           n_individuals=20,
                           n_trials=50, 
                           n_items=3,
                           error_weight=0,
                           value_range=[1, 10],
                           seed=None)

make_model_args = dict(kind='hierarchical',
                       t0_val=0,
                       error_weight=0)

fit_args = dict(method='MCMC',
                step=pm.Metropolis,
                draws=20000,
                tune=180000,
                cores=1,
                chains=2,
                progressbar=False)

wrapper = partial(pr.recover_and_save_hierarchical,
                  output_folder=partial_output_folder,
                  label=label,
                  simulate_group_args=simulate_group_args,
                  make_model_args=make_model_args,
                  fit_args=fit_args)

In [8]:
from multiprocessing import Pool

n_cores = 22
p = Pool(n_cores)

output = p.map(wrapper, pr.generate_hierarchical_parameter_sets(parameter_info))

Generating hierarchical model for 20 subjects...
Generating hierarchical model for 20 subjects...
Generating hierarchical model for 20 subjects...
Generating hierarchical model for 20 subjects...
Generating hierarchical model for 20 subjects...
Generating hierarchical model for 20 subjects...
Generating hierarchical model for 20 subjects...
Generating hierarchical model for 20 subjects...
Generating hierarchical model for 20 subjects...
Generating hierarchical model for 20 subjects...
Generating hierarchical model for 20 subjects...
Generating hierarchical model for 20 subjects...
Generating hierarchical model for 20 subjects...
Generating hierarchical model for 20 subjects...
Generating hierarchical model for 20 subjects...
Generating hierarchical model for 20 subjects...
Generating hierarchical model for 20 subjects...
Generating hierarchical model for 20 subjects...
Generating hierarchical model for 20 subjects...
Generating hierarchical model for 20 subjects...
Generating hierarchi

INFO (theano.gof.compilelock): Waiting for existing lock by process '73716' (I am process '73734')
INFO (theano.gof.compilelock): To manually release the lock, delete /Users/felixmolter/.theano/compiledir_Darwin-17.7.0-x86_64-i386-64bit-i386-3.6.8-64/lock_dir
INFO (theano.gof.compilelock): Waiting for existing lock by process '73716' (I am process '73725')
INFO (theano.gof.compilelock): To manually release the lock, delete /Users/felixmolter/.theano/compiledir_Darwin-17.7.0-x86_64-i386-64bit-i386-3.6.8-64/lock_dir
INFO (theano.gof.compilelock): Waiting for existing lock by process '73716' (I am process '73715')
INFO (theano.gof.compilelock): To manually release the lock, delete /Users/felixmolter/.theano/compiledir_Darwin-17.7.0-x86_64-i386-64bit-i386-3.6.8-64/lock_dir
INFO (theano.gof.compilelock): Waiting for existing lock by process '73716' (I am process '73728')
INFO (theano.gof.compilelock): To manually release the lock, delete /Users/felixmolter/.theano/compiledir_Darwin-17.7.0-x

Fitting 1 model(s) using MCMC...
  Fitting model 1 of 1...


INFO (theano.gof.compilelock): Waiting for existing lock by process '73734' (I am process '73725')
INFO (theano.gof.compilelock): To manually release the lock, delete /Users/felixmolter/.theano/compiledir_Darwin-17.7.0-x86_64-i386-64bit-i386-3.6.8-64/lock_dir
INFO (theano.gof.compilelock): Waiting for existing lock by process '73734' (I am process '73715')
INFO (theano.gof.compilelock): To manually release the lock, delete /Users/felixmolter/.theano/compiledir_Darwin-17.7.0-x86_64-i386-64bit-i386-3.6.8-64/lock_dir
INFO (theano.gof.compilelock): Waiting for existing lock by process '73734' (I am process '73728')
INFO (theano.gof.compilelock): To manually release the lock, delete /Users/felixmolter/.theano/compiledir_Darwin-17.7.0-x86_64-i386-64bit-i386-3.6.8-64/lock_dir
INFO (theano.gof.compilelock): Waiting for existing lock by process '73734' (I am process '73726')
INFO (theano.gof.compilelock): To manually release the lock, delete /Users/felixmolter/.theano/compiledir_Darwin-17.7.0-x

INFO (theano.gof.compilelock): Waiting for existing lock by process '73725' (I am process '73722')
INFO (theano.gof.compilelock): Waiting for existing lock by process '73725' (I am process '73729')
INFO (theano.gof.compilelock): Waiting for existing lock by process '73725' (I am process '73731')
INFO (theano.gof.compilelock): To manually release the lock, delete /Users/felixmolter/.theano/compiledir_Darwin-17.7.0-x86_64-i386-64bit-i386-3.6.8-64/lock_dir
INFO (theano.gof.compilelock): To manually release the lock, delete /Users/felixmolter/.theano/compiledir_Darwin-17.7.0-x86_64-i386-64bit-i386-3.6.8-64/lock_dir
INFO (theano.gof.compilelock): To manually release the lock, delete /Users/felixmolter/.theano/compiledir_Darwin-17.7.0-x86_64-i386-64bit-i386-3.6.8-64/lock_dir
INFO (theano.gof.compilelock): Waiting for existing lock by process '73725' (I am process '73718')
INFO (theano.gof.compilelock): To manually release the lock, delete /Users/felixmolter/.theano/compiledir_Darwin-17.7.0-x

...

/!\ Automatically setting parameter precision...


The gelman-rubin statistic is larger than 1.4 for some parameters. The sampler did not converge.
The estimated number of effective samples is smaller than 200 for some parameters.


/!\ Automatically setting parameter precision...


The gelman-rubin statistic is larger than 1.05 for some parameters. This indicates slight problems during sampling.
The estimated number of effective samples is smaller than 200 for some parameters.


/!\ Automatically setting parameter precision...


## Combine results

In [9]:
filenames = [file for file in listdir(partial_output_folder)
             if file.endswith('.csv')]

partial_recoveries = []

for file in filenames:
    partial = pd.read_csv(join(partial_output_folder, file))
    partial_recoveries.append(partial)

parameter_recovery = pd.concat(partial_recoveries).sort_values('index').reset_index(drop=True)
del partial_recoveries
parameter_recovery.to_csv(join(output_folder, 'parameter-recovery_{}.csv'.format(label)))
parameter_recovery.head()

,index,v_level,gamma_level,s_level,tau_level,v_gen,gamma_gen,s_gen,tau_gen,v_rec,...,gamma_rec,gamma_rec_hpd_lower,gamma_rec_hpd_upper,s_rec,s_rec_hpd_lower,s_rec_hpd_upper,tau_rec,tau_rec_hpd_lower,tau_rec_hpd_upper,converged
0,0,low,low,low,low,0.000033,-0.8,0.0067,0.215,0.000052,...,-0.651710,-0.757312,-0.542939,0.006963,0.006433,0.007473,0.309947,0.229713,0.396732,True
1,1,low,low,low,medium,0.000033,-0.8,0.0067,0.675,0.000049,...,-0.654757,-0.727285,-0.578590,0.007142,0.006521,0.007733,0.760665,0.639258,0.881303,True
2,2,low,low,low,high,0.000033,-0.8,0.0067,1.135,0.000052,...,-0.635629,-0.701134,-0.567091,0.007329,0.006770,0.007903,1.229496,1.012402,1.438628,True
3,3,low,low,medium,low,0.000033,-0.8,0.0095,0.215,0.000050,...,-0.658418,-0.819515,-0.504449,0.009670,0.008936,0.010349,0.295966,0.170887,0.407633,True
4,4,low,low,medium,medium,0.000033,-0.8,0.0095,0.675,0.000052,...,-0.630054,-0.713714,-0.545045,0.009753,0.009103,0.010387,0.748147,0.539259,0.977551,True
